In [1]:
%%shell
jupyter nbconvert --to html "/content/APS360Project.ipynb"

UsageError: Cell magic `%%shell` not found.


In [2]:
import os
import random
import numpy as np
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.io as torchio
from   torch.utils.data.sampler import SubsetRandomSampler
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF
import torchvision.datasets as tds
import matplotlib.pyplot as plt
#import pandas as pd
import PIL
import shutil

# Mounting Google Drive
# from google.colab import drive
# drive.mount('/content/drive')

LOCAL = True
SHOW_RESULTS = False # !LOCAL
# set to true when animation images have changed to rebuild
# all the 128x128 crops. After first run, this can be set back to false
REFRESH_CROPS = False


In [3]:
class Autoencoder(nn.Module):
    # 32 is what the paper starts with
    def __init__(self, startOutCh = 32, depthRatio = 16 / 9):
        super(Autoencoder, self).__init__()

        # Added Name
        self.name = "AEhalfmdatar3_sod{0}_odr{1:.2f}".format(startOutCh, depthRatio)
        self.startOutCh = startOutCh
        
        # Values
        startOutCh2 = int(startOutCh * depthRatio)
        startOutCh3 = int(startOutCh2 * depthRatio)
        startOutCh4 = int(startOutCh3 * depthRatio)
        
        # Convolution Layers
        self.Conv2D_1 = nn.Conv2d(in_channels = 3, out_channels = startOutCh, kernel_size = 3, stride = 1, padding = 1)
        self.Conv2D_2 = nn.Conv2d(startOutCh, startOutCh2, 3, 1, 1)
        self.Conv2D_3 = nn.Conv2d(startOutCh2, startOutCh3, 3, 1, 1)
        self.Conv2D_4 = nn.Conv2d(startOutCh3, startOutCh4, 3, 1, 1)
        
        self.Conv2D_T1 = nn.ConvTranspose2d(startOutCh4, startOutCh3, 3, 1, 1)
        self.Conv2D_T2 = nn.ConvTranspose2d(startOutCh3, startOutCh2, 3, 1, 1)
        self.Conv2D_T3 = nn.ConvTranspose2d(startOutCh2, startOutCh, 3, 1, 1)
        self.Conv2D_T4 = nn.ConvTranspose2d(startOutCh, 3, 3, 1, 1)
        
        # Pooling & Up-Scaling Layers
        self.Pooling_1 = nn.MaxPool2d(4, 4)
        self.Expanding_1 = nn.UpsamplingNearest2d(scale_factor = 4)
        self.ReLU = nn.ReLU()
        self.Sigmoid = nn.Sigmoid()      

    def forward(self, x):   
        x1 = self.Conv2D_1(x)
        x = self.ReLU(self.Pooling_1(x1))
        x2 = self.Conv2D_2(x)
        x = self.ReLU(self.Pooling_1(x2))
        x3 = self.Conv2D_3(x)
        x = self.ReLU(self.Pooling_1(x3))
        
        x4 = self.Conv2D_4(x)
        # x = self.ReLU(x4)
        x = self.Conv2D_T1(x4)
        x = self.ReLU(x) + self.Conv2D_T1(x4)
        
        x = self.Expanding_1(self.Conv2D_T2(x))
        x = self.ReLU(x) + self.Conv2D_T2(x3)
        x = self.Expanding_1(self.Conv2D_T3(x))
        x = self.ReLU(x) + self.Conv2D_T3(x2)
        x = self.Expanding_1(self.Conv2D_T4(x))
        x = self.ReLU(x) + self.Conv2D_T4(x1)
        return x

In [4]:
!nvidia-smi

use_cuda = True

model = Autoencoder()

if use_cuda and torch.cuda.is_available():
  model.cuda()
  print('CUDA is available!  Training on GPU ...')
else:
  print('CUDA is not available.  Training on CPU ...')

Thu Aug 10 08:55:48 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 536.99                 Driver Version: 536.99       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3050 ...  WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   57C    P5              11W /  80W |    267MiB /  4096MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [5]:
torch.cuda.max_memory_allocated()

1795584

In [6]:
def get_model_name(name, batch_size, learning_rate, epoch=None):
    """ Generate a name for the model consisting of all the hyperparameter values"""
    if epoch is None:
        return "model_{0}_bs{1}_lr{2}".format(name, batch_size, learning_rate)
    else:
        return "model_{0}_bs{1}_lr{2}_epoch{3}".format(name, batch_size, learning_rate, epoch)

def plt_img_tensor(tensor):
    t = torch.transpose(torch.transpose(tensor, 0, 2), 0, 1)
    plt.imshow(t.detach().cpu())
    plt.show()

def save_img_tensor(tensor, save_dir, img_name):
    t = torch.clamp(tensor * 255., min=0., max=255.).byte().detach().cpu()
    torchio.write_png(t, save_dir + img_name + ".png", compression_level=0)


def train(model, model_dir, result_dir, train_ds, valid_ds, num_epochs=5, batch_size=1, lr=1e-3):
    torch.manual_seed(42)
    criterion = nn.L1Loss() # L1 Loss is used for model updates
    criterion_compair = nn.MSELoss() # mean square error loss for standarization
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)

    train_loader = torch.utils.data.DataLoader(train_ds, batch_size=batch_size, shuffle=True)

    #outputs = []
    for epoch in range(num_epochs):
        for data in train_loader:
            
            noisy, truth = data            
            recon = model(noisy)
            loss = criterion(recon, truth)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            
        loss_oimg = criterion_compair(recon, truth)
        print('Epoch:{}, MSE Loss:{:.4f}'.format(epoch+1, float(loss_oimg)))

        for i in range(recon.size()[0]):
            save_img_tensor(truth[i], result_dir, str(epoch + 1) + "_" + str(i) + "_truth")
            save_img_tensor(recon[i], result_dir, str(epoch + 1) + "_" + str(i) + "_recon")
            save_img_tensor(noisy[i], result_dir, str(epoch + 1) + "_" + str(i) + "_noisy")

        if SHOW_RESULTS:
            plt_img_tensor(truth[0])
            plt_img_tensor(recon[0])

        # Save the current model (checkpoint) to a file
        model_path = get_model_name(model.name, batch_size, lr, epoch + 1)
        torch.save(model.state_dict(), os.path.join(model_dir, model_path))

        #outputs.append((epoch, img, recon),)
    #return outputs

#plt.imshow(np.transpose(final_recon[0][0].detach().numpy()), (1, 2, 0))

In [7]:
# Address for Datasets within the Drive
BasePath = "data/" if LOCAL else "/content/drive/MyDrive/data/"
BaseAnimPath = BasePath + "baseline_data/animations/"
truthPaths = [BaseAnimPath + "anim1/4096",
              BaseAnimPath + "anim2/4096",
              BaseAnimPath + "anim3/4096",
              BaseAnimPath + "anim5/4096"]
noisyPaths = [BaseAnimPath + "anim1/1",
              BaseAnimPath + "anim2/1",
              BaseAnimPath + "anim3/1",
              BaseAnimPath + "anim5/1"]

def get_io_paths(noisy_dirs, truth_dirs):
  """
        noisy_dirs: List of noisy image directories.
        truth_dirs: List of truth image directories.
  """
  assert len(noisy_dirs) == len(truth_dirs)

  paths = []
  for i in range(len(noisy_dirs)):
    nfiles = [os.path.join(noisy_dirs[i], f) for f in os.listdir(noisy_dirs[i])]
    tfiles = [os.path.join(truth_dirs[i], f) for f in os.listdir(truth_dirs[i])]
    nfiles.sort()
    tfiles.sort()
    # assert len(nfiles) == len(tfiles)
    paths.extend(zip(nfiles, tfiles))

  return paths

# this is the only thing that works to avoid running out of RAM in Colab
class ImagesDataset(torch.utils.data.Dataset):
    def __init__(self, io_paths, transform=None):
        """
        io_paths: list of tuples of input-output image paths.
        transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.paths = io_paths
        self.transform = transform

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
      noisy_img = torchio.read_image(self.paths[idx][0], torchio.ImageReadMode.RGB).to(torch.float)
      truth_img = torchio.read_image(self.paths[idx][1], torchio.ImageReadMode.RGB).to(torch.float)

      if use_cuda and torch.cuda.is_available():
        noisy_img = noisy_img.cuda()
        truth_img = truth_img.cuda()
         
      noisy_img /= 255.
      truth_img /= 255.

      if (self.transform):
        noisy_img = self.transform(noisy_img)
        truth_img = self.transform(truth_img)

      return [noisy_img, truth_img]


def write_crops(img, img_idx, dir):
  crop_paths = []
  for j in range(64):
    crop_idx_x = j % 8
    crop_idx_y = int(j / 8)
    img_crop = TF.crop(img, 128 * crop_idx_y, 128 * crop_idx_x, 128, 128)
    crop_path = dir + str(img_idx) + "_" + str(j) + ".png"
    torchio.write_png(img_crop, crop_path, compression_level=0)
    crop_paths.append(crop_path)
  return crop_paths


train_split = 0.7
valid_split = 0.15
test_split = 0.15

crop_dir = BaseAnimPath + "crops/"
noisy_crop_dir = crop_dir + "noisy/"
truth_crop_dir = crop_dir + "truth/"

if REFRESH_CROPS and os.path.exists(crop_dir):
  shutil.rmtree(crop_dir)

if os.path.exists(crop_dir):
  paths = get_io_paths([noisy_crop_dir], [truth_crop_dir])
else:
  paths = get_io_paths(noisyPaths, truthPaths)
  new_paths = []
  os.mkdir(crop_dir)
  os.mkdir(noisy_crop_dir)
  os.mkdir(truth_crop_dir)

  for i in range(len(paths)):
    noisy_img = torchio.read_image(paths[i][0], torchio.ImageReadMode.RGB)
    truth_img = torchio.read_image(paths[i][1], torchio.ImageReadMode.RGB)
    ncrop_paths = write_crops(noisy_img, i, noisy_crop_dir)
    tcrop_paths = write_crops(truth_img, i, truth_crop_dir)
    new_paths.extend(zip(ncrop_paths, tcrop_paths))

  paths = new_paths
  
random.shuffle(paths)
train_eidx = int(len(paths) * train_split)
valid_eidx = int(len(paths) * (train_split + valid_split))

#trans256 = transforms.CenterCrop(size = 256)
train_ds = ImagesDataset(paths[:train_eidx])#, transform=trans256)
valid_ds = ImagesDataset(paths[train_eidx:valid_eidx])#, transform=trans256)
test_ds = ImagesDataset(paths[valid_eidx:])#, transform=trans256)

BatchSize = 16
LearningRate = 5e-4


model = Autoencoder().cuda()
model_dir = BasePath + "nnmodel/{}/".format(model.name)
if not os.path.exists(model_dir):
  os.mkdir(model_dir)
  
result_dir = model_dir + get_model_name(model.name, BatchSize, LearningRate) + "_results/"
if not os.path.exists(result_dir):
    os.mkdir(result_dir)

fpaths = open(os.path.join(model_dir, "paths.txt"), "w")
fpaths.write("split={0},{1},{2}".format(train_split, valid_split, test_split) + "\n")
for p in paths:
  fpaths.write(str(p) + "\n")
fpaths.close()

#train_loader = torch.utils.data.DataLoader(train_ds, batch_size=1, shuffle=True, num_workers=2)
#plt.imshow(np.transpose(np.array(next(iter(train_loader))[0][0]), (1, 2, 0)))

In [18]:
model.load_state_dict(torch.load("data/nnmodel/AEhalfmdatar3_sod32_odr1.78/model_AEhalfmdatar3_sod32_odr1.78_bs16_lr0.0005_epoch75"))

<All keys matched successfully>

In [21]:
demo_path = r"E:\Maya\UofT\8th sem\aps360\project\APS360_raytracing_denoising\localver\test2\breakfast32.png"
gt_path = r"E:\Maya\UofT\8th sem\aps360\project\APS360_raytracing_denoising\localver\test2\breakfast8192.png"
gblur_path = r"E:\Maya\UofT\8th sem\aps360\project\APS360_raytracing_denoising\localver\test2\breakfast32_guassianblur.png"

demo_ds = ImagesDataset([(demo_path, gt_path), (gblur_path, gt_path),])
#save_img_tensor(next(iter(demo_ds))[0], "E:\\Maya\\UofT\\8th sem\\aps360\\project\\APS360_raytracing_denoising\\localver\\", "demo1cropped.png")

demo_dl = torch.utils.data.DataLoader(demo_ds, batch_size=1)

demo_dat = next(iter(demo_dl))
demo_noisy, demo_gt = demo_dat
demo_dat2 = next(iter(demo_dl))
demo_gblur, _ = demo_dat2

demo_recon = model(demo_noisy)

cmp = nn.MSELoss()
print("MSE loss recon", float(cmp(demo_recon, demo_gt)))
print("MSE loss guassian blur k5", float(cmp(demo_gblur, demo_gt)))

save_img_tensor(demo_recon[0], "E:\\Maya\\UofT\\8th sem\\aps360\\project\\APS360_raytracing_denoising\\localver\\test2\\", "breakfast_recon")

torch.Size([1, 3, 1024, 1024])
MSE loss recon 0.012149984948337078
MSE loss guassian blur k5 0.0750330463051796


In [42]:
train(model, model_dir, result_dir, train_ds, valid_ds, batch_size = BatchSize, num_epochs = 500, lr = LearningRate)

Epoch:1, MSE Loss:0.0003
Epoch:2, MSE Loss:0.0008
Epoch:3, MSE Loss:0.0008
Epoch:4, MSE Loss:0.0014
Epoch:5, MSE Loss:0.0004
Epoch:6, MSE Loss:0.0004
Epoch:7, MSE Loss:0.0022
Epoch:8, MSE Loss:0.0005
Epoch:9, MSE Loss:0.0007
Epoch:10, MSE Loss:0.0012
Epoch:11, MSE Loss:0.0008
Epoch:12, MSE Loss:0.0019
Epoch:13, MSE Loss:0.0005
Epoch:14, MSE Loss:0.0010
Epoch:15, MSE Loss:0.0011
Epoch:16, MSE Loss:0.0010
Epoch:17, MSE Loss:0.0008
Epoch:18, MSE Loss:0.0008
Epoch:19, MSE Loss:0.0013
Epoch:20, MSE Loss:0.0010
Epoch:21, MSE Loss:0.0009
Epoch:22, MSE Loss:0.0006
Epoch:23, MSE Loss:0.0005
Epoch:24, MSE Loss:0.0027
Epoch:25, MSE Loss:0.0013
Epoch:26, MSE Loss:0.0018
Epoch:27, MSE Loss:0.0015
Epoch:28, MSE Loss:0.0021
Epoch:29, MSE Loss:0.0006
Epoch:30, MSE Loss:0.0027
Epoch:31, MSE Loss:0.0009
Epoch:32, MSE Loss:0.0003
Epoch:33, MSE Loss:0.0005
Epoch:34, MSE Loss:0.0006
Epoch:35, MSE Loss:0.0008
Epoch:36, MSE Loss:0.0006
Epoch:37, MSE Loss:0.0014
Epoch:38, MSE Loss:0.0009
Epoch:39, MSE Loss:0.

KeyboardInterrupt: 